# Training The Transfer Learning Models
## InceptionV3 based transfer learning technique for comparison with SimpleNet
The notebook contains code used for training a transfer learning method using InceptionV3 model. 
The model classifies preprocessed retinal OCT scans into three categories: urgent referral, routine referral
and normal. **5-fold cross validation** was used to present an accurate estimate of the efficacy of the model 
on unseen data. The notebooks were run on the kaggle platform for training purposes. **Since the 5-folds
and initial weights of the deep neural net are random, the accuracies might not match with the accuracies
given in the publication.**

### Table of Contents
* [1. Importing all modules](#1)
* [2. Loading train and test data](#2)
* [3. Generator Classes using Sequence class](#3)
* [4. Functions for building and loading the transfer learning model](#4)
* [5. Functions for training and cross-validation](#5)
* [6. Training the model](#6)
* [7. Performance metrics of InceptionV3 based model](#7)

<a id="1"></a>
### 1. Importing all modules

In [1]:

import os
import keras
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
import pandas as pd
from keras.preprocessing import image
from keras.utils import Sequence
from cv2 import * #Import functions from OpenCV
import cv2
import glob
from skimage.transform import resize
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import model_from_json
import json
from statistics import mean



Using TensorFlow backend.


<a id="2"></a>
### 2. Loading train and test data

Training data is loaded from the initial k-fold split across all the four models for accurate judgement.



In [2]:
# X and y stores the images as arrays and the their target categories respectively

X = np.genfromtxt('../input/final-paper-kfold/X.out', dtype='unicode')
y = np.genfromtxt('../input/final-paper-kfold/y.out')

#reading test images, because can't use validation generator here, as that limilts the time

t_img = []
t_y=[]
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/CNV/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(2)    
    
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/DME/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(2)    
    
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/DRUSEN/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(1)    
    
for images in glob.glob("../input/no-med-filt-224/no_median_filter_224/no_median_filter_224/test/NORMAL/*.jpeg"):
    n = resize(imread(images), (224, 224,3))
    t_img.append(n)
    t_y.append(0)    

t_img = np.asarray(t_img)
t_y = np.asarray(t_y)


<a id="3"></a>
### 3. Generator Classes using Sequence class
Since our training dataset is large, we would use the Sequence class to help us feed images in batches for training.
Different generator classes are defined for training on four folds and cross-validation on the remaining fold.

In [3]:
class My_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        batch_y = self.labels[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name), (224, 224,3))
               for file_name in batch_x])
        y = np.array(batch_y)
        return X,y
    
class My_valid_Generator(Sequence):

    def __init__(self, image_filenames, batch_size):
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name), (224, 224,3))
               for file_name in batch_x])
        return X
    

<a id="4"></a>
### 4. Functions for building and loading the transfer learning model

For each cross-validation build_model() builds the initial InceptionV3 based model framework whereas load_ith_model() 
loads the ith cross-validation model from memory for testing and validation.  

In [4]:
# for the first 22 epochs

def build_model():
    #load InceptionV3 without dense layer and with theano dim ordering
    base_model = InceptionV3(weights='../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top = False, input_shape = (224,224,3))
    for layer in base_model.layers:
        # batch-normalization layers were trained on previous imagenet images
        # so we are training batch-normalization layers again for our specific dataset
        if layer.name[0:10]!='batch_norm':
            layer.trainable = False 
    num_classes = 3

    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation = 'softmax')(x)

    #create graph of your new model
    model = Model(input = base_model.input, output = predictions)

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def load_ith_model(i):
    json_file = open('model-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('model-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [5]:
# this is for the next 28 epochs

def build_model_old():
    json_file = open('../input/model-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('../input/model-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def train_save_validate_old(i):
    batch_size = 64
    train_index = np.genfromtxt('../input/final-paper-kfold/train_index-'+str(i)+'.out').astype(int)
    test_index = np.genfromtxt('../input/final-paper-kfold/test_index-'+str(i)+'.out').astype(int)
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]

    trainGenerator = My_Generator(trainData,trainLabels,batch_size)
    valGenerator = My_valid_Generator(testData,400)
    
    model = build_model_old()
    
    model.fit_generator(
            generator = trainGenerator,
            steps_per_epoch=(len(trainData)//batch_size),
            epochs=28, verbose = 2, class_weight = [0.70596402, 4.19022748, 0.74357918])
    
    model_json = model.to_json()
    with open('model-'+str(i)+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('model-'+str(i)+'.h5')
    
    t_y = np.argmax(testLabels,axis=1)    
    ans = np.argmax(model.predict_generator(valGenerator),axis =1)
    
    # performance metrics  
    return performance_metrics(t_y,ans)


<a id="5"></a>
### 5. Functions for training and cross-validation
For each cross-validation i, train_save_validate() trains the model, saves model-i and saves the validation performance
metrics on validation fold. 

In [6]:

def train_save_validate(i):
    batch_size = 64
    train_index = np.genfromtxt('../input/final-paper-kfold/train_index-'+str(i)+'.out').astype(int)
    test_index = np.genfromtxt('../input/final-paper-kfold/test_index-'+str(i)+'.out').astype(int)
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]

    trainGenerator = My_Generator(trainData,trainLabels,batch_size)
    valGenerator = My_valid_Generator(testData,400)
    
    model = build_model()
    if i ==1:
        model.summary()
        
    model.fit_generator(
            generator = trainGenerator,
            steps_per_epoch=(len(trainData)//batch_size),
            epochs=22, verbose = 2, class_weight = [0.70596402, 4.19022748, 0.74357918])
    
    model_json = model.to_json()
    with open('model-'+str(i)+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('model-'+str(i)+'.h5')
    
    t_y = np.argmax(testLabels,axis=1)    
    ans = np.argmax(model.predict_generator(valGenerator),axis =1)
    
    # performance metrics  
    return performance_metrics(t_y,ans)

**perfromance_metrics()** derives all performance measures from the confusion matrix whereas **print_performance_average_each_model()** prints the validation metrics in a readable format.

In [7]:
def performance_metrics(test_y,ans):
    acc = accuracy_score(test_y,ans)
    print('Accuracy = ',acc)
    target_names = ['CLass Normal', 'CLass Early', 'Class Late']
    
    # computing all metrics via the confusion matrix
    cm = confusion_matrix(test_y, ans) 
    sens0 = cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2])
    spec0 = (cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])/((cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])+(cm[1,0]+cm[2,0]))

    sens1 = cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2])
    spec1 = (cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])/((cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])+(cm[0,1]+cm[2,1]))

    sens2 = cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2])
    spec2 = (cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])/((cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])+(cm[0,2]+cm[1,2]))
  
    rep=classification_report(test_y, ans, target_names=target_names, digits = 4, labels=range(0,3),output_dict=True)
    return [acc,sens0,spec0,sens1,spec1,sens2,spec2],cm,rep

# method for printing performance 
def print_performance_average_each_model(p,i):
    a = list(p[i][2].keys())
    b = list(p[i][2][a[0]].keys())

    s = np.zeros(7)
    pn = 0
    pe = 0
    pl =0
    fn =0
    fe = 0
    fl = 0
    rn = 0
    re = 0
    rl = 0
    for j in range(i,i+1):
        s = s + p[j][0]
        pn = pn +  p[j][2][a[0]][b[0]]
        pe = pe +  p[j][2][a[1]][b[0]]
        pl = pl +  p[j][2][a[2]][b[0]]
        fn = fn + p[j][2][a[0]][b[2]]
        fe = fe + p[j][2][a[1]][b[2]]
        fl = fl +  p[j][2][a[2]][b[2]]
        rn = rn +  p[j][2][a[0]][b[1]]
        re = re +  p[j][2][a[1]][b[1]]
        rl = rl +  p[j][2][a[2]][b[1]]
    print("\n\n###### Metrics for Fold ",i, " ######" )
    print('\nacc = ',round(s[0]*100,2))
    print("              sensitivity  specificity      precision      f1-score")
    print('\nClass Normal: ',round(s[1]*100,2), ",       ", round(s[2]*100,2), ",       ", round(pn*100,2),",       ", round(fn*100,2))
    print('\nClass Early:  ',round(s[3]*100,2), ",       ", round(s[4]*100,2), ",       ", round(pe*100,2),",       ", round(fe*100,2))
    print('\nClass Late:   ',round(s[5]*100,2), ",       ", round(s[6]*100,2), ",       ", round(pl*100,2),",       ", round(fl*100,2))

    print('\n\n##### Average Performance Metrics for Fold ', i, ' #####')
    avg_prec = round(p[j][2][a[5]][b[0]]*100,2)
    avg_sens = round(np.sum([s[1]*p[j][2][a[0]][b[3]],
                      s[3]*p[j][2][a[1]][b[3]],
                      s[5]*p[j][2][a[2]][b[3]]])*(100/p[j][2][a[5]][b[3]]),2)
    avg_spec = round(np.sum([s[2]*p[j][2][a[0]][b[3]],
                      s[4]*p[j][2][a[1]][b[3]],
                      s[6]*p[j][2][a[2]][b[3]]])*(100/p[j][2][a[5]][b[3]]),2)
    avg_f1 = round(p[j][2][a[5]][b[2]]*100,2)
    acc = round(s[0]*100,2)
    print('\nprecision: ',avg_prec,'sensitivity: ',avg_sens, ", specificity: ",avg_spec, ", f1-score: ", avg_f1)
    return [avg_prec, avg_sens, avg_spec, avg_f1, acc]


 <a id="6"></a>
### 6. Training the InceptionV3 model
This cell is not executed in this notebook as each training will yield different results. Kindly send me an email if you need the trained weights used in the publication.

Each cell executes a part of the cross-validation because each cell in kaggle had a maximum running time limit. This notebook shows the first 22 epochs of execution, the next 28 epochs were run on a separate notebook (with adequately modified functions to load and train the model) and is not shown due to redundancy. Kindly send me an email if you need the other notebook. If you have the computing power to run it all in one go, please change of epochs from 22 to 50 in the train_save_validate(i) method.


In [8]:
valid_perf = {}

# the first 22 epochs
for i in range(1,2):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate(i)


====== K Fold Validation step => 1/5 =======
Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# fold 2 first 22 epochs
for i in range(2,3):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate(i)


====== K Fold Validation step => 2/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1107s - loss: 1.5312 - acc: 0.8613
Epoch 2/22
 - 1022s - loss: 1.4100 - acc: 0.8941
Epoch 3/22
 - 1010s - loss: 1.3560 - acc: 0.9049
Epoch 4/22
 - 1019s - loss: 1.3467 - acc: 0.9081
Epoch 5/22
 - 1054s - loss: 1.3277 - acc: 0.9120
Epoch 6/22
 - 1017s - loss: 1.3375 - acc: 0.9118
Epoch 7/22
 - 1031s - loss: 1.3210 - acc: 0.9148
Epoch 8/22
 - 1022s - loss: 1.3168 - acc: 0.9154
Epoch 9/22
 - 1041s - loss: 1.3252 - acc: 0.9147
Epoch 10/22
 - 1017s - loss: 1.3228 - acc: 0.9152
Epoch 11/22
 - 1014s - loss: 1.3179 - acc: 0.9162
Epoch 12/22
 - 1008s - loss: 1.3283 - acc: 0.9152
Epoch 13/22
 - 1051s - loss: 1.3195 - acc: 0.9164
Epoch 14/22
 - 1020s - loss: 1.3215 - acc: 0.9160
Epoch 15/22
 - 1022s - loss: 1.3063 - acc: 0.9177
Epoch 16/22
 - 1014s - loss: 1.3320 - acc: 0.9158
Epoch 17/22
 - 1037s - loss: 1.3068 - acc: 0.9177
Epoch 18/22
 - 1009s - loss: 1.3149 - acc: 0.9176
Epoch 19/22
 - 1004s - loss: 1.3136 - acc: 0.9173
Epoch 20/22
 - 1000s - loss: 1.3143 - acc: 0.9173
Epoch 21/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# fold 3 first 22 epochs

for i in range(3,4):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    train_save_validate(i)


====== K Fold Validation step => 3/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1049s - loss: 1.5195 - acc: 0.8631
Epoch 2/22
 - 986s - loss: 1.4105 - acc: 0.8949
Epoch 3/22
 - 978s - loss: 1.3637 - acc: 0.9043
Epoch 4/22
 - 974s - loss: 1.3368 - acc: 0.9097
Epoch 5/22
 - 1010s - loss: 1.3299 - acc: 0.9110
Epoch 6/22
 - 977s - loss: 1.3349 - acc: 0.9121
Epoch 7/22
 - 971s - loss: 1.3274 - acc: 0.9141
Epoch 8/22
 - 972s - loss: 1.3243 - acc: 0.9144
Epoch 9/22
 - 1010s - loss: 1.3387 - acc: 0.9136
Epoch 10/22
 - 975s - loss: 1.3201 - acc: 0.9158
Epoch 11/22
 - 972s - loss: 1.3143 - acc: 0.9166
Epoch 12/22
 - 969s - loss: 1.3270 - acc: 0.9155
Epoch 13/22
 - 1008s - loss: 1.3554 - acc: 0.9131
Epoch 14/22
 - 977s - loss: 1.3332 - acc: 0.9152
Epoch 15/22
 - 978s - loss: 1.3331 - acc: 0.9157
Epoch 16/22
 - 969s - loss: 1.3174 - acc: 0.9167
Epoch 17/22
 - 1009s - loss: 1.3259 - acc: 0.9165
Epoch 18/22
 - 977s - loss: 1.3126 - acc: 0.9177
Epoch 19/22
 - 976s - loss: 1.3133 - acc: 0.9175
Epoch 20/22
 - 970s - loss: 1.3215 - acc: 0.9170
Epoch 21/22
 - 1013s - l

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# fold 3 next 28 epochs
for i in range(3,4):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate_old(i)


====== K Fold Validation step => 3/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/28
 - 1071s - loss: 1.3181 - acc: 0.9174
Epoch 2/28
 - 1006s - loss: 1.3188 - acc: 0.9174
Epoch 3/28
 - 1014s - loss: 1.3219 - acc: 0.9172
Epoch 4/28
 - 1010s - loss: 1.3175 - acc: 0.9175
Epoch 5/28
 - 1048s - loss: 1.3127 - acc: 0.9180
Epoch 6/28
 - 1003s - loss: 1.3148 - acc: 0.9178
Epoch 7/28
 - 1004s - loss: 1.3094 - acc: 0.9181
Epoch 8/28
 - 1007s - loss: 1.3116 - acc: 0.9179
Epoch 9/28
 - 1045s - loss: 1.3150 - acc: 0.9177
Epoch 10/28
 - 1004s - loss: 1.3143 - acc: 0.9178
Epoch 11/28
 - 1007s - loss: 1.3116 - acc: 0.9182
Epoch 12/28
 - 1006s - loss: 1.3167 - acc: 0.9177
Epoch 13/28
 - 1039s - loss: 1.3126 - acc: 0.9181
Epoch 14/28
 - 1015s - loss: 1.3097 - acc: 0.9181
Epoch 15/28
 - 999s - loss: 1.3163 - acc: 0.9179
Epoch 16/28
 - 1008s - loss: 1.3154 - acc: 0.9179
Epoch 17/28
 - 1043s - loss: 1.3021 - acc: 0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# fold 4 first 22 epochs

for i in range(4,5):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    train_save_validate(i)


====== K Fold Validation step => 4/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1028s - loss: 1.5136 - acc: 0.8636
Epoch 2/22
 - 964s - loss: 1.4179 - acc: 0.8952
Epoch 3/22
 - 947s - loss: 1.3713 - acc: 0.9042
Epoch 4/22
 - 933s - loss: 1.3481 - acc: 0.9092
Epoch 5/22
 - 962s - loss: 1.3338 - acc: 0.9120
Epoch 6/22
 - 934s - loss: 1.3336 - acc: 0.9128
Epoch 7/22
 - 926s - loss: 1.3433 - acc: 0.9124
Epoch 8/22
 - 925s - loss: 1.3293 - acc: 0.9146
Epoch 9/22
 - 957s - loss: 1.3239 - acc: 0.9154
Epoch 10/22
 - 926s - loss: 1.3248 - acc: 0.9158
Epoch 11/22
 - 930s - loss: 1.3236 - acc: 0.9156
Epoch 12/22
 - 922s - loss: 1.3290 - acc: 0.9156
Epoch 13/22
 - 953s - loss: 1.3304 - acc: 0.9156
Epoch 14/22
 - 928s - loss: 1.3277 - acc: 0.9161
Epoch 15/22
 - 928s - loss: 1.3177 - acc: 0.9170
Epoch 16/22
 - 928s - loss: 1.3174 - acc: 0.9169
Epoch 17/22
 - 956s - loss: 1.3164 - acc: 0.9172
Epoch 18/22
 - 926s - loss: 1.3213 - acc: 0.9169
Epoch 19/22
 - 930s - loss: 1.3289 - acc: 0.9162
Epoch 20/22
 - 928s - loss: 1.3191 - acc: 0.9171
Epoch 21/22
 - 951s - loss: 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# fold 4 next 28 epochs

for i in range(4,5):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate_old(i)


====== K Fold Validation step => 4/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/28
 - 1053s - loss: 1.3186 - acc: 0.9174
Epoch 2/28
 - 1002s - loss: 1.3128 - acc: 0.9179
Epoch 3/28
 - 1004s - loss: 1.3171 - acc: 0.9175
Epoch 4/28
 - 1000s - loss: 1.3182 - acc: 0.9174
Epoch 5/28
 - 1040s - loss: 1.3276 - acc: 0.9169
Epoch 6/28
 - 1017s - loss: 1.3203 - acc: 0.9173
Epoch 7/28
 - 995s - loss: 1.3137 - acc: 0.9181
Epoch 8/28
 - 1005s - loss: 1.3075 - acc: 0.9185
Epoch 9/28
 - 1041s - loss: 1.3264 - acc: 0.9171
Epoch 10/28
 - 998s - loss: 1.3154 - acc: 0.9180
Epoch 11/28
 - 1001s - loss: 1.3204 - acc: 0.9174
Epoch 12/28
 - 994s - loss: 1.3174 - acc: 0.9177
Epoch 13/28
 - 1033s - loss: 1.3159 - acc: 0.9179
Epoch 14/28
 - 997s - loss: 1.3214 - acc: 0.9176
Epoch 15/28
 - 1028s - loss: 1.3174 - acc: 0.9177
Epoch 16/28
 - 1021s - loss: 1.3135 - acc: 0.9179
Epoch 17/28
 - 1064s - loss: 1.3179 - acc: 0.91

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
# fold 5 first 22 epochs

for i in range(5,6):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i] = train_save_validate(i)


====== K Fold Validation step => 5/5 =======
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/22
 - 1129s - loss: 1.5045 - acc: 0.8632
Epoch 2/22
 - 1026s - loss: 1.4128 - acc: 0.8942
Epoch 3/22
 - 1023s - loss: 1.3668 - acc: 0.9044
Epoch 4/22
 - 1014s - loss: 1.3414 - acc: 0.9092
Epoch 5/22
 - 1096s - loss: 1.3231 - acc: 0.9127
Epoch 6/22
 - 1067s - loss: 1.3193 - acc: 0.9138
Epoch 7/22
 - 1039s - loss: 1.3175 - acc: 0.9143
Epoch 8/22
 - 1034s - loss: 1.3159 - acc: 0.9152
Epoch 9/22
 - 1073s - loss: 1.3163 - acc: 0.9157
Epoch 10/22
 - 1025s - loss: 1.3172 - acc: 0.9158
Epoch 11/22
 - 1028s - loss: 1.3092 - acc: 0.9167
Epoch 12/22
 - 1029s - loss: 1.3118 - acc: 0.9166
Epoch 13/22
 - 1061s - loss: 1.3095 - acc: 0.9170
Epoch 14/22
 - 1025s - loss: 1.3044 - acc: 0.9178
Epoch 15/22
 - 1007s - loss: 1.3134 - acc: 0.9172
Epoch 16/22
 - 1020s - loss: 1.3277 - acc: 0.9157
Epoch 17/22
 - 1056s - loss: 1.3483 - acc: 0.9139
Epoch 18/22
 - 1016s - loss: 1.3387 - acc: 0.9152
Epoch 19/22
 - 1019s - loss: 1.3268 - acc: 0.9164
Epoch 20/22
 - 1031s - loss: 1.3184 - acc: 0.9172
Epoch 21/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


<a id="7"></a>
### 7. Performance metrics of InceptionV3 based model

In [15]:
# all performance metrics for each fold and their averages

perf = np.zeros(5)

for i in range(1,6):
    perf += print_performance_average_each_model(sim,i)
perf = perf/5

print('\n#### AVERAGE PERFORMANCE OVER ALL FOLDS ####\n\n\nprecision: ',round(perf[0],2),'sensitivity: ',round(perf[1],2), ", specificity: ",round(perf[2],2), ", f1-score: ", round(perf[3],2))   
print('\n[',round(perf[0],2),', ',round(perf[1],2), ", ",round(perf[2],2), ", ", round(perf[3],2),"]")   
print('\n overall accuracy of model = ', round(perf[4],2))






###### Metrics for Fold  1  ######

acc =  90.52
              sensitivity  specificity      precision      f1-score

Class Normal:  98.83 ,        93.94 ,        93.49 ,        96.08

Class Early:   0.0 ,        100.0 ,        0.0 ,        0.0

Class Late:    97.73 ,        88.57 ,        87.62 ,        92.4


##### Average Performance Metrics for Fold  1  #####

precision:  83.45 sensitivity:  90.52 , specificity:  91.99 , f1-score:  86.82


###### Metrics for Fold  2  ######

acc =  90.32
              sensitivity  specificity      precision      f1-score

Class Normal:  97.81 ,        94.82 ,        94.51 ,        96.13

Class Early:   0.0 ,        100.0 ,        0.0 ,        0.0

Class Late:    98.51 ,        87.48 ,        86.25 ,        91.97


##### Average Performance Metrics for Fold  2  #####

precision:  83.31 sensitivity:  90.32 , specificity:  91.98 , f1-score:  86.62


###### Metrics for Fold  3  ######

acc =  90.56
              sensitivity  specificity      precisio